In [3]:
import numpy as np

import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.20.0


In [5]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep='\t')

CougsInAzure	westus2	CougsInAzure


In [6]:
experiment_name = 'TestModel1'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

creating new compute target...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-11T14:17:59.928000+00:00', 'errors': None, 'creationTime': '2021-02-11T14:17:57.630059+00:00', 'modifiedTime': '2021-02-11T14:18:13.027998+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [8]:
import os
script_folder = os.path.join(os.getcwd(), "TestModel1")
os.makedirs(script_folder, exist_ok=True)

In [9]:
%%writefile $script_folder/train.py

import os
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import joblib
from azureml.core import Workspace, Dataset, Run
import azureml.core
import random

def IsPositive(r):
    return True if r[-1] else False
def IsNegative(r):
    return True if not r[-1] else False

subscription_id = '9d0dfa04-d2f8-4521-b945-b3a7dbf43946'
resource_group = 'CougsInAzure'
workspace_name = 'CougsInAzure'

ws = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(ws, name='Alpha_Featurized')
df = dataset.to_pandas_dataframe()

df_list = df.values.tolist()


positive_dataset = list(filter(IsPositive, df_list))
negative_dataset = list(filter(IsNegative, df_list))

random.shuffle(positive_dataset)
random.shuffle(negative_dataset)

dataset = []

l = len(negative_dataset) if len(negative_dataset) < len(positive_dataset) else len(positive_dataset)

for i in range(0,l):
    dataset.append(positive_dataset[i])
    dataset.append(negative_dataset[i])

X = list(map(lambda x: x[:-1], dataset))
y = list(map(lambda x: x[-1], dataset))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)

run = Run.get_context()

dt = DecisionTreeClassifier().fit(X_train, y_train)

clf = DecisionTreeClassifier().fit(X_train, y_train)

print('Predict the test set')
y_hat = clf.predict(X_test)

# calculate accuracy on the prediction
acc = np.average(y_hat == y_test)
print('Accuracy is', acc)

#run.log('regularization rate', np.float(args.reg))
run.log('accuracy', np.float(acc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=clf, filename='outputs/TestModel1.pkl')

Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/azuretester1/code/Users/capstonesipsml/Deployments/TestModel1/train.py


In [10]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
    
env = Environment("Test-env1")
cd = CondaDependencies.create(pip_packages=['azureml-defaults'], conda_packages = ['scikit-learn'])

env.python.conda_dependencies = cd

env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210104.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "Test-env1",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
  

In [12]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=script_folder,
                      script='train.py',
                      compute_target=compute_target,
                      environment=env)

In [13]:
run = exp.submit(config=src)
run

Experiment,Id,Type,Status,Details Page,Docs Page
TestModel1,TestModel1_1613053122_7688188c,azureml.scriptrun,Failed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
model = run.register_model(model_name='TestModel1', model_path='outputs/TestModel1.pkl')
print(model.name, model.id, model.version, sep='\t')

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/TestModel1.pkl in the set of files uploaded to the run: []
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path outputs/TestModel1.pkl in the set of files uploaded to the run: []\n                See https://aka.ms/run-logging for more details."
    }
}

## deployment step

In [15]:
from azureml.core.conda_dependencies import CondaDependencies

# Add the dependencies for your model
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = 'service_files/env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

Saved dependency info in service_files/env.yml


In [13]:
from azureml.core.model import InferenceConfig

classifier_inference_config = InferenceConfig(runtime= "python",
                                              source_directory = 'service_files',
                                              entry_script="score.py",
                                              conda_file="env.yml")

In [14]:
from azureml.core.compute import ComputeTarget, AksCompute

cluster_name = 'aks-cluster2'
compute_config = AksCompute.provisioning_configuration(location='westus')
production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
production_cluster.wait_for_completion(show_output=True)

ComputeTargetException: ComputeTargetException:
	Message: Received bad response from Resource Provider:
Response Code: 400
Headers: {'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Length': '321', 'Content-Type': 'application/json; charset=utf-8', 'Expires': '-1', 'x-ms-correlation-request-id': '18e0e15c-f956-482c-9708-82e8a1be614e', 'x-ms-ratelimit-remaining-subscription-writes': '1199', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-response-type': 'standard', 'x-ms-request-id': '|00-041d0f5aa535dd48a5f0bfa09137bea7-8c03e31983ded542-00.3e3e08a3_', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.049', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'x-ms-routing-request-id': 'WESTUS2:20210121T194315Z:18e0e15c-f956-482c-9708-82e8a1be614e', 'Date': 'Thu, 21 Jan 2021 19:43:14 GMT'}
Content: b'{"error":{"code":"NotSupported","message":"A compute with the same name already exists. Updating property: provisioningState 

In [16]:
from azureml.core.webservice import AksWebservice

classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 1)

In [21]:
from azureml.core.model import Model

model = ws.models['TestModel1']
service = Model.deploy(workspace=ws,
                       name = 'alpha-prototype',
                       models = [model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......
Succeeded
AKS service creation operation finished, operation "Succeeded"
